In [5]:
# load sample original and transcribed questions to compare
data_path = '/ivi/ilps/personal/svakule/spoken_qa/'
dataset_name = 'WD18_entities/'
qo_path = data_path + dataset_name + "valid_original.jsonl"  # original text questions
qt_path = data_path + dataset_name + "valid_wav2vec2-base-960h.jsonl"  # questions transcribed from synthethised speech

In [14]:
import json

def load(path):
    qs = {}
    with open(path) as fin:
        for line in fin:
            q = json.loads(line)
            qs[q['_id']] = q['text']
    return qs

qos = load(qo_path)
qts = load(qt_path)

In [23]:
_id = '2'

# tokenize
from transformers import AutoTokenizer, AutoModel

# you can switch the model to the original "distilbert-base-uncased" to see that the usage example then breaks and the score ordering is reversed :O
#pre_trained_model_name = "distilbert-base-uncased"
pre_trained_model_name = "sebastian-hofstaetter/distilbert-dot-tas_b-b256-msmarco"

tokenizer = AutoTokenizer.from_pretrained(pre_trained_model_name) 
bert_model = AutoModel.from_pretrained(pre_trained_model_name)

print(qos[_id])
query_input_qo = tokenizer(qos[_id], return_tensors="pt")
print("Query Tokenized:", tokenizer.convert_ids_to_tokens(query_input_qo["input_ids"][0]))

print(qts[_id])
query_input_qt = tokenizer(qts[_id], return_tensors="pt")
print("Query Tokenized:", tokenizer.convert_ids_to_tokens(query_input_qt["input_ids"][0]))

in what french city did antoine de févin die 
Query Tokenized: ['[CLS]', 'in', 'what', 'french', 'city', 'did', 'antoine', 'de', 'fe', '##vin', 'die', '[SEP]']
in what french city did untwinde feaven die
Query Tokenized: ['[CLS]', 'in', 'what', 'french', 'city', 'did', 'un', '##t', '##wind', '##e', 'fe', '##ave', '##n', 'die', '[SEP]']


In [24]:
query_encoded_qo = bert_model(**query_input_qo)[0][:,0,:].squeeze(0)
query_encoded_qt = bert_model(**query_input_qt)[0][:,0,:].squeeze(0)

score0 = query_encoded_qo.dot(query_encoded_qo)
print("Score: ", float(score0))
score = query_encoded_qo.dot(query_encoded_qt)
print("Score: ", float(score))

Score:  114.05656433105469
Score:  103.06715393066406
